In [42]:
from importlib import reload
from nesi_unbalanced import unbalanced2
from libdatasets import *
import libdatasets
from libactive import active_split
from dotenv import load_dotenv; load_dotenv()
from tabulate import tabulate
import numpy as np
import nesi_unbalanced
import scipy
import traceback

In [2]:
reload(libdatasets); from libdatasets import *

In [54]:
reload(nesi_unbalanced); from nesi_unbalanced import matrix, unbalanced2
for name, dataset in matrix['datasets']:
    try:
        X_labelled, X_unlabelled, Y_labelled, Y_oracle, X_test, Y_test = active_split(
            *dataset(), mutator=partial(unbalanced2, amount=0.7), test_size=0.5, labeled_size=10, shuffle=True, random_state=np.random
        )
    except (ValueError, IndexError) as e:
        print(name, "could not be split:", e)
        continue
    if isinstance(X_labelled, scipy.sparse.csr_matrix):
        X = scipy.sparse.vstack((X_labelled, X_unlabelled))
    else:
        X = np.concatenate((X_labelled, X_unlabelled))
    y = np.concatenate((Y_labelled, Y_oracle))
    print(name, "has a split", y.shape[0], Y_test.shape[0])
    
    
    print(tabulate(zip(np.unique(y), [f"{x:.0%}" for x in np.unique(y, return_counts=True)[1]/y.shape[0]], [f"{x:.0%}" for x in np.unique(Y_test, return_counts=True)[1]/Y_test.shape[0]]), headers=["Class", "Train", "Test"]))
    print()

avila has a split 6205 10434
Class    Train    Test
-------  -------  ------
A        70%      40%
B        0%       0%
C        0%       1%
D        2%       3%
E        5%       11%
F        10%      19%
G        2%       4%
H        2%       5%
I        4%       8%
W        0%       0%
X        2%       5%
Y        1%       3%

swarm has a split 8700 12008
  Class  Train    Test
-------  -------  ------
      0  30%      51%
      1  70%      49%

sensorless has a split 3896 29255
  Class  Train    Test
-------  -------  ------
      1  3%       9%
      2  70%      9%
      3  3%       9%
      4  3%       9%
      5  3%       9%
      6  3%       9%
      7  3%       9%
      8  3%       9%
      9  3%       9%
     10  3%       9%
     11  3%       9%

anuran has a split 2481 3598
Class                   Train    Test
----------------------  -------  ------
AdenomeraAndre          5%       9%
AdenomeraHylaedactylus  70%      48%
Ameeregatrivittata      4%       8%
HylaMinuta     

C:\Users\Zac\Programming\python\research\libactive.py:71: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if klass not in Y_labelled:


### Chosen datasets

* avila,
* smartphone
* sensorless

### Accuracy Differences

In [46]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from time import monotonic
reload(nesi_unbalanced); from nesi_unbalanced import matrix, unbalanced2
results = []
for (name, dataset) in matrix['datasets']:
    print(name)
    X, y = dataset()
    
    try:
        X_labelled, X_unlabelled, Y_labelled, Y_oracle, X_test, Y_test = active_split(
            *dataset(), mutator=partial(unbalanced2, amount=0.7), test_size=0.5, labeled_size=10, shuffle=True, random_state=np.random
        )
    except (ValueError, IndexError) as e:
        print(name, "could not be split:", e)
        traceback.print_exc()
        continue
    if isinstance(X_labelled, scipy.sparse.csr_matrix):
        X = scipy.sparse.vstack((X_labelled, X_unlabelled))
    else:
        X = np.concatenate((X_labelled, X_unlabelled))
    y = np.concatenate((Y_labelled, Y_oracle))
    
    clf = SVC(kernel='linear', probability=True)

    clf.fit(X_labelled, Y_labelled)
    start = clf.score(X_test, Y_test)
    start_t = monotonic()
    clf.fit(X_unlabelled[:1000], Y_oracle[:1000])
    time = monotonic() - start_t
    final = clf.score(X_test, Y_test)
    results.append([name, start, final, time])

avila


C:\Users\Zac\Programming\python\research\libactive.py:71: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if klass not in Y_labelled:


swarm
sensorless
anuran


C:\Users\Zac\Programming\python\research\libactive.py:71: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if klass not in Y_labelled:


In [47]:
results = np.array(results)
print(tabulate(np.hstack((results, np.expand_dims(results[:,2].astype(float)-results[:,1].astype(float), axis=1))), headers=["Name", "Initial acc", "Final acc", "Time", "Diff"]))

Name          Initial acc    Final acc    Time      Diff
----------  -------------  -----------  ------  --------
avila            0.222638     0.518497   0.203  0.29586
swarm            0.648235     0.960859   3.14   0.312625
sensorless       0.247479     0.719467   0.296  0.471988
anuran           0.774597     0.913285   0.047  0.138688


### Stable seed test

In [62]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state
from time import monotonic
reload(nesi_unbalanced); from nesi_unbalanced import matrix, unbalanced2
results = []
for (name, dataset) in matrix['datasets']:
    print(name)
    X, y = dataset()
    
    for i in range(11):
        try:
            X_labelled, X_unlabelled, Y_labelled, Y_oracle, X_test, Y_test = active_split(
                *dataset(), mutator=partial(unbalanced2, amount=0.7), test_size=0.5, labeled_size=10, shuffle=True, random_state=check_random_state(i)
            )
        except Exception as e:
            print(f"  {i} FAILED: {e}")
        else:
            print(f"  {i} OK")


avila
A
  0 FAILED: index 0 is out of bounds for axis 0 with size 0
A
  1 OK
A
  2 OK
A
  3 OK
A
  4 OK
A
  5 OK
A
  6 OK
A
  7 OK
A
  8 OK
A
  9 OK
A
  10 OK
swarm
1
  0 OK
1
  1 OK
1
  2 OK
0
  3 OK
1
  4 OK
0
  5 OK
0
  6 OK
0
  7 OK
0
  8 OK
0
  9 OK
0
  10 OK
sensorless
10
  0 OK
8
  1 OK
10
  2 OK
4
  3 OK
8
  4 OK
9
  5 OK
1
  6 OK
1
  7 OK
8
  8 OK
5
  9 OK
3
  10 OK
anuran
AdenomeraHylaedactylus
  0 OK
AdenomeraHylaedactylus
  1 OK
AdenomeraHylaedactylus
  2 OK
AdenomeraHylaedactylus
  3 OK
AdenomeraHylaedactylus
  4 OK
AdenomeraHylaedactylus
  5 OK
AdenomeraHylaedactylus
  6 OK
AdenomeraHylaedactylus
  7 OK
AdenomeraHylaedactylus
  8 OK
AdenomeraHylaedactylus
  9 OK
AdenomeraHylaedactylus
  10 OK


C:\Users\Zac\Programming\python\research\libactive.py:71: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if klass not in Y_labelled:
